# What is a TfidfVectorizer?



# What is a PassiveAggressiveClassifier?


In [5]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [6]:
notfake = pd.read_csv('notfake_newsdataset.csv')

In [7]:
fake=pd.read_csv('fake_newsdataset.csv')

In [8]:
notfake['target']=1
fake['target']=0
df = pd.concat([notfake , fake] , ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [9]:
df.head(20)

,title,text,subject,date,target
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0
5,"As private lawyer, Trump high court pick was f...",WASHINGTON (Reuters) - As a lawyer in private ...,politicsNews,"February 1, 2017",1
6,Yemeni Salafist imam killed in Aden: sources,ADEN (Reuters) - A Salafist imam was shot dead...,worldnews,"October 28, 2017",1
7,FBI says witnesses in U.S. probe into Malaysia...,KUALA LUMPUR (Reuters) - Potential witnesses t...,worldnews,"September 6, 2017",1
8,An Easy To Read Chart Shows How Bernie Sanders...,The goal of socialism is communism. -Vladimi...,politics,"Feb 24, 2016",0
9,MMA FIGHTER JAKE SHIELDS Embarrasses Cowards I...,Opposing views and beliefs has much of this co...,politics,"Feb 4, 2017",0


In [10]:
df.shape

(44898, 5)

In [11]:
def summary(df):
    sum_df = pd.DataFrame(df.dtypes, columns=["dtypes"])
    sum_df["missing"] = df.isna().sum()
    sum_df["missing %"] = (df.isna().sum())/ len(df)
    sum_df["uniques"] = df.nunique().values
    sum_df["count"] = df.count().values
    return sum_df.sort_values("missing %", ascending=False)
summary(df)

,dtypes,missing,missing %,uniques,count
title,object,0,0.0,38729,44898
text,object,0,0.0,38646,44898
subject,object,0,0.0,8,44898
date,object,0,0.0,2397,44898
target,int64,0,0.0,2,44898


In [12]:
df.drop(['date','subject'],axis='columns',inplace=True)

In [13]:
df.columns

Index(['title', 'text', 'target'], dtype='object')

In [14]:
#DataFlair - Split the dataset
X = df.text
X.head(10)

0    Donald Trump s White House is in chaos, and th...
1    Now that Donald Trump is the presumptive GOP n...
2    Mike Pence is a huge homophobe. He supports ex...
3    SAN FRANCISCO (Reuters) - California Attorney ...
4    Twisted reasoning is all that comes from Pelos...
5    WASHINGTON (Reuters) - As a lawyer in private ...
6    ADEN (Reuters) - A Salafist imam was shot dead...
7    KUALA LUMPUR (Reuters) - Potential witnesses t...
8     The goal of socialism is communism.  -Vladimi...
9    Opposing views and beliefs has much of this co...
Name: text, dtype: object

In [15]:
y = df.target
y.head(10)

0    0
1    0
2    0
3    1
4    0
5    1
6    1
7    1
8    0
9    0
Name: target, dtype: int64

In [27]:
import sweetviz as sv
my_report=sv.analyze(df)
my_report.show_html()

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [87]:
#DataFlair - Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

#DataFlair - Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(X_train) 
tfidf_test=tfidf_vectorizer.transform(X_test)

In [89]:
#DataFlair - Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

# #DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.38%


In [91]:
#DataFlair - Build confusion matrix
confusion_matrix(y_test,y_pred, labels=[0,1])

array([[4589,   28],
       [  28, 4335]], dtype=int64)